In [4]:
import os, importlib, sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

%load_ext autoreload 

In [5]:
# Common paths
BASE_PATH = os.path.join(os.getcwd(), "..", "..")
MODULE_PATH = os.path.join(BASE_PATH, "modules")
DS_PATH = os.path.join(BASE_PATH, "datasets")

In [6]:
sys.path.append(MODULE_PATH)

import mp.MomentPropagation as mp
importlib.reload(mp)

import data.mnist as mnist_loader
importlib.reload(mnist_loader)

<module 'data.mnist' from '/home/exleonem/Desktop/workspace/thesis/wp/notebooks/model/../../modules/data/mnist.py'>

In [7]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import  Conv2D, MaxPool2D, Dropout, Flatten, Dense, Softmax

In [8]:
gpus = tf.config.experimental.list_physical_devices("GPU")
cpus = tf.config.experimental.list_physical_devices("CPU")

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            
        logical_gpus = tf.config.experimental.list_logical_devices("GPU")
        print(len(gpus), "Physical GPU's, ", len(logical_gpus), "Logical GPU's")
        
    except RuntimeError as e:
        print(e)
        
elif cpus:
    try:
        logical_cpus = tf.config.experimental.list_logical_devices("CPU")
        print(len(cpus), "Physical CPU,", len(logical_cpus), "Logical CPU")
        
    except RuntimeError as e:
        print(e)
        
        
tfk = tf.keras

1 Physical GPU's,  1 Logical GPU's


In [9]:
input_shape = (28, 28, 1)
model = Sequential([
    Conv2D(128, 4,activation="relu", input_shape=input_shape),
    MaxPool2D(),
    Dropout(.2),
    Conv2D(64, 3, activation="relu"),
    MaxPool2D(),
    Dropout(.2),
    Flatten(),
    Dense(512, activation="relu"),
    Dense(256, activation="relu"),
    Dense(128, activation="relu"),
    Dense(1, activation="sigmoid")
])

model.build(input_shape)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 25, 25, 128)       2176      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 64)        73792     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0

In [10]:
%autoreload 2
import active_learning as active
importlib.reload(active)

<module 'active_learning' from '/home/exleonem/Desktop/workspace/thesis/wp/notebooks/model/../../modules/active_learning/__init__.py'>

In [12]:
import data.mnist as mnist_loader
importlib.reload(mnist_loader)

# Load Data
mnist_path = os.path.join(DS_PATH, "mnist")
inputs, targets = mnist_loader.load(mnist_path)

# Select only first and second class
selector = (targets==0) | (targets==1)
new_inputs = inputs[selector].astype("float32")/255.0
new_targets = targets[selector]

# Create splits
x_train, x_test, y_train, y_test = train_test_split(new_inputs, new_targets)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test)

In [13]:
inputs = np.expand_dims(x_train, axis=-1)

metrics = ["binary_accuracy"]
model.compile(optimizer="sgd", loss="binary_crossentropy", metrics=metrics)
model.fit(x=inputs, y=y_train, batch_size=20, epochs=5)

Epoch 1/5
555/555 [==============================] - 7s 12ms/step - loss: 0.1412 - binary_accuracy: 0.9647
Epoch 2/5
555/555 [==============================] - 2s 3ms/step - loss: 0.0107 - binary_accuracy: 0.9966
Epoch 3/5
555/555 [==============================] - 2s 3ms/step - loss: 0.0074 - binary_accuracy: 0.9976
Epoch 4/5
555/555 [==============================] - 2s 3ms/step - loss: 0.0064 - binary_accuracy: 0.9978
Epoch 5/5
555/555 [==============================] - 2s 3ms/step - loss: 0.0048 - binary_accuracy: 0.9983


In [14]:
new_inputs.shape

(14780, 28, 28)

In [15]:
new_inputs[:, None, ...].shape

(14780, 1, 28, 28)

In [16]:
sample_input = np.random.randn(10, 28, 28, 1)
sample_input.shape

(10, 28, 28, 1)

In [26]:
%autoreload 2
import bayesian 
from bayesian import McDropout, MomentPropagation

In [34]:
%autoreload 2

from acl import ActiveLearning
from active_learning import TrainConfig, Config, Metrics, aggregates_per_key

import bayesian 
from bayesian import McDropout, MomentPropagation


train_config = TrainConfig(
    batch_size=2,
    epochs=1
)

acq_config = Config(
    name="std_mean",
    pseudo=True
)

model_name = "mp"
acq_name = "max_entropy"
dp_model = McDropout(model)
mp_model = MomentPropagation(model)


active_learning = ActiveLearning(
    mp_model, 
    np.expand_dims(new_inputs, axis=-1), labels=new_targets, 
    train_config=train_config,
    acq_name=acq_name,
    debug=False
)

history = active_learning.start(step_size=40)


# Save history
METRICS_PATH = os.path.join(BASE_PATH, "metrics")
metrics = Metrics(METRICS_PATH, keys=["iteration", "train_time", "query_time", "loss"])
metrics.write(model_name + "_" + acq_name, history)

  0%|          | 0/278 [00:00<?, ?it/s]

/home/exleonem/Desktop/workspace/thesis/wp/modules/logs/acl.log
/home/exleonem/Desktop/workspace/thesis/wp/modules/active_learning/../logs/acf.log


  1%|          | 3/278 [00:30<47:08, 10.29s/it]


KeyboardInterrupt: 